# InvokeAI in Google Colab

This is a tool to use Google colab to run the AI image generation tool: Invokeai (https://invoke-ai.github.io/InvokeAI/).
This automatically builds itself and can import custom models, It can connect to Google drive to save your images.
It also has the option of running entirely from Google drive to vastly speed up the time it takes to start, However, over 20Gb of Google Drive space is required! However it changes subsequent startup times from 15-20 mins to 5-6 mins.

Make sure to enable GPU, Edit > Notebook Settings > Hardware accelerator > GPU

To start, Click "Runtime" > "Run All". Alternaivly you can click the "play" button on each step below one after the other, No need to wait for the previous steps to finish as they will join a queue.<br>


In [ ]:
#@title 1. Configuration { display-mode: "form" }
#@markdown #Instance Type
#@markdown **Google_Drive** = Stores ALL files in your Google drive, This can take over 20Gb of space (dependent on your settings), but will start up much faster as it does not download invokeai at each run. To re-run the full install please use the "Rebuild" feature in advanced settings, or alternativly delete the whole install directory. <br>
#@markdown **Persistent** = Stores only your images in Google drive, in a folder called "/invokeai/invokeaiOutputV3".<br>
#@markdown **Temporary** (NOT recomended) = Everything is stored in the runtime and is removed when the runtime ends or crashes, make sure to download your images! <br><br>
#@markdown **Rough Startup times:** <br>
#@markdown Google_Drive, First run, or any "Persistent/Temporary" run = 15-20 mins <br>
#@markdown Google_Drive, Subsiquent runs, 6-7 mins.
Type = "Google_Drive" #@param ['Google_Drive','Persistent','Temporary'] {type:"string"}

#@markdown ---

#@markdown #Connection Type.
#@markdown Localtunnel: Stable once connected, but sometimes has issues.<br>
#@markdown Serveo: less stable, requires no setup. As an alternative to Localtunnel<br>
#@markdown NGROK: (Recomended) Highly stable but needs a little setting up, An NGROK token is required, sign up for free and get one here: https://dashboard.ngrok.com/get-started/your-authtoken Once you have the token, please put it in below.<br>
connection_type = "Localtunnel"  #@param ["Serveo","Localtunnel","NGROK"]
ngrok_token = "None" #@param ['None'] {allow-input: true}

#@markdown ---

#@markdown #NSFW Checker - This has been moved into the interface.<br>
#@markdown This checks outputs for "Not Safe For Work" content and obscures the image. This is not 100% reliable.<br>
#@markdown To activate this, in the top right corner choose the three lines > Settings > Generations > and Toggle "Enable NSFW checker" as desired.

#nsfw_checker = "Enabled" #@param ['Enabled' ,'Disabled'] {type:"string"}




In [ ]:
#@title 2. Models { display-mode: "form" }

#@markdown Skip Default Models.<br><br>
#@markdown If you are not intending to use them, it is HIGHLY recomended to skip the default models. as this will save 20+GB of downloads and about 10 mins of loading.<br>
skip_default_models = "No" #@param ["Yes","No"]

#@markdown With all custom model types if you are using a "Google_Drive" type install, you can run this once to generate the folder structure and manually upload models into google drive into the folder /invokeai/invokeaiV3/autoimport/RELAVENT_FOLDER (e.g. /autoimport/vae) or you can use one of the below methods to add models into any install type.<br>

#@markdown Custom Diffuser Model
custom_diffuser_hugging_face_id = "None" #@param ["None","kebab111/crosskemono", "dreamlike-art/dreamlike-photoreal-2.0", "stablediffusionapi/disney-pixal-cartoon"] {allow-input: true}
#@markdown Custom Safetensor or Checkpoint models <br>
custom_model_url = "None" #@param ["None","https://huggingface.co/ffggggfg/Crosskemono/resolve/main/crosskemonoFurryModel_crosskemono25.safetensors"] {allow-input: true}
custom_vae_url = "None" #@param ["None","https://huggingface.co/ffggggfg/Crosskemono/resolve/main/Crosskemono2.vae.pt"] {allow-input: true}

# #@markdown If you have selected a custom diffuser model it will be auto imported and it is selectable from the dropdown menu in app. <br>
# #@markdown Custom .safetensor or .checkpoint models are stored in /content/invokeai/CustomModels and must be manually added via the GUI after it starts. Your Custom VAE will be stored in /content/invokeai/CustomVae <br>

#@markdown ---

#@markdown LoRA <br>
#@markdown To import a LoRA please please input a hugging face repo ID or the URL to your LoRA file.<br>
HuggingFace_LoRA_Repo_ID = "None" #@param ["None", "artificialguybr/LogoRedmond-LogoLoraForSDXL"] {allow-input: true}
custom_LoRA_url = "None" #@param ["None"] {allow-input: true}


#@markdown ControlNet <br>
#@markdown To import a ControlNet please input a hugging face repo ID or the URL to your ControlNet file.<br>
HuggingFace_ControlNet_Repo_ID = "None" #@param ["None", "CrucibleAI/ControlNetMediaPipeFace"] {allow-input: true}
custom_ControlNet_url = "None" #@param ["None"] {allow-input: true}


In [ ]:
#@title 3. Advanced Options { display-mode: "form" }

#@markdown #Version
#@markdown By default this application uses "The latest official release of InvokeAI" however if you wish to use a specific version please enter its version code below.<br>
#@markdown Please be aware, If you are using a "Google_Drive" version of this program. DOWNGRADING is not officially supported. It may work, it may not, If this does break InvokeAI please DELETE the whole folder /Google_Drive/InvokeAI/invokeaiV3 from https://drive.google.com/, If required please backup your "Autoimport" folder. <br>
#@markdown Some recomeneded older versions can be located in the dropdown. but any version 3.0.2a1 or newer should work.
Version = "Default" #@param ["Default","3.1.0","3.0.2.post1","3.0.2a1"] {allow-input: true}

#@markdown #Rebuild
#@markdown If you are having any issues with a GoogleDrive install of InvokeAI but want to keep your imported models and settings, set this to "Yes" to force a rebuild of the app. <br>
#@markdown Additionally some settings such as "Skip defult models" are ignored on subsiquent runs, use this to run those steps again. <br>

Rebuild = "No" #@param ["Yes","No"]


#@markdown ---
#@markdown #Textual Inversion Training mode - This is VERY much in the eirly devlopemnt stage - All options in step "6" MUST be configured.
#@markdown This mode does not run InvokeAI, it instead uses its "Textual inversion Training" feature to generate a .pt model <br>
#@markdown it is HIGHLY recomended that you run this from Google Drive. <br>
#@markdown Place the images you wish to train with in "invokeaiOutputV3/text-inversion-training-data/MODEL_NAME" The exact path will vary depending on the mode you choose, the MODEL_NAME is set in step 6 below.<br>
#@markdown InvokeAI Guide: https://invoke-ai.github.io/InvokeAI/features/TRAINING/
ti_mode = "No" #@param ['Yes','No'] {allow-input: false}



In [ ]:
#@title 4. Build and Configure, Invokeai V3 { display-mode: "form" }
#@markdown This takes about 15 mins to finish + some extra time to download custom models. <br>
#@markdown Version 3 Downloads all models during the build phase.<br>
#@markdown It will auto add all models to the app.

from IPython.display import clear_output

#Set up temporary storage if running in "Temporary" mode.
if Type == "Temporary":
    file_path = '/content/invokeai'
    outpath = '/content/invokeai/output'
    noUpdate = '/content/invokeai/noUpdate'
    import os
    if not os.path.exists(file_path):
      os.makedirs(file_path)

# Mount and set up Google drive if running in "Persistent" mode.
if Type == "Persistent":
  file_path = '/content/invokeai'
  outpath = '/content/drive/MyDrive/invokeai/invokeaiOutputV3'
  noUpdate = '/content/invokeai/noUpdate'
  import os
  from google.colab import drive
  if not os.path.exists(file_path):
    drive.mount('/content/drive')
    if not os.path.exists(file_path):
      os.makedirs(file_path)

# Mount and set up Google drive if running in "Google_Drive" mode.
if Type == "Google_Drive":
  file_path = '/content/drive/MyDrive/invokeai/invokeaiV3'
  outpath = '/content/drive/MyDrive/invokeai/invokeaiOutputV3'
  noUpdate = '/content/drive/MyDrive/invokeai/invokeaiV3/noUpdate'

  import os
  from google.colab import drive
  if not os.path.exists(file_path):
    drive.mount('/content/drive')
    if not os.path.exists(file_path):
      os.makedirs(file_path)

#Action the rebuild flag.
if Rebuild == "Yes":
  !sudo rm -R {noUpdate}

#Install Python + Pip
!sudo apt-get update -y
!sudo apt-get install python3.11 python3-pip python3.11-distutils python3.11-dev libpython3.11-dev python3.11-venv -y

#Update pip
%cd {file_path}
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3 get-pip.py --force-reinstall
!python -m pip install --upgrade pip
!pip install --upgrade setuptools

#Install some Google colab dependencies.
!python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

#Create InvokeAI root
import os
os.environ['INVOKEAI_ROOT'] = file_path
if not os.path.exists(file_path):
  os.makedirs(invokeai_root)

#Create the virtual environment + Download default Models
%cd {file_path}

#On 2nd run, Do an "upgrade" to get system variables to load quickly.
if os.path.exists(noUpdate):
  !python -m venv .venv --prompt InvokeAI
  if Version=="Default":
    !source .venv/bin/activate; python -m pip install InvokeAI[xformers] --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117 --upgrade
  if Version != "Default":
    !source .venv/bin/activate; python -m pip install InvokeAI[xformers]=={Version} --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117 --upgrade


#PIP First time install of InvokeAI.
if not os.path.exists(noUpdate):
  !python -m venv .venv --prompt InvokeAI
  !source .venv/bin/activate; python -m pip install --upgrade pip

  if Version=="Default":
    !source .venv/bin/activate; python -m pip install InvokeAI[xformers] --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117
  if Version != "Default":
    !source .venv/bin/activate; python -m pip install InvokeAI[xformers]=={Version} --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117

  !mkdir {outpath}
  !cd {file_path}

  if skip_default_models == "Yes":
    !source .venv/bin/activate; invokeai-configure --yes --skip-sd-weights
  if skip_default_models == "No":
    !source .venv/bin/activate; invokeai-configure --yes

  !mkdir {noUpdate}

## Edit invokeai.yaml

#Toggle the NSFW checker - Archived
#if nsfw_checker == "Disabled":
#  !sed -i 's/nsfw_checker: true/nsfw_checker: false/' invokeai.yaml
#if nsfw_checker == "Enabled":
#  !sed -i 's/nsfw_checker: false/nsfw_checker: true/' invokeai.yaml

#Download custom checkpoint models
if custom_model_url != "None":
  %cd {file_path}/autoimport/main
  !wget -nc {file_path}/autoimport/main {custom_model_url}

#Download custom VAE
if custom_vae_url != "None":
  %cd {file_path}/autoimport/vae
  !wget -nc {file_path}/autoimport/vae {custom_vae_url}

#Import Diffuser Model
if custom_diffuser_hugging_face_id != "None":
  %cd {file_path}/autoimport/main
  !git clone --depth 1 --verbose --progress https://huggingface.co/{custom_diffuser_hugging_face_id}

#Import LoRA
if custom_LoRA_url != "None":
  %cd {file_path}/autoimport/lora
  !wget -nc {file_path}/autoimport/lora {custom_LoRA_url}
if HuggingFace_LoRA_Repo_ID != "None":
  %cd {file_path}/autoimport/lora
  !git clone --depth 1 --verbose --progress https://huggingface.co/{HuggingFace_LoRA_Repo_ID}

#Import ControlNet
if custom_ControlNet_url != "None":
  %cd {file_path}/autoimport/controlnet
  !wget -nc {file_path}/autoimport/controlnet {custom_ControlNet_url}
if HuggingFace_ControlNet_Repo_ID != "None":
  %cd {file_path}/autoimport/controlnet
  !git clone --depth 1 --verbose --progress https://huggingface.co/{HuggingFace_ControlNet_Repo_ID}

%cd {file_path}

#Clear Output
clear_output()

In [ ]:
#@title 5. Start InvokeAI. { display-mode: "form" }
if ti_mode == "No":
  #@markdown ### Starting the App
  #@markdown This step takes about 30 seconds to generate your URL but 2-3 mins before it will work fully!. <br>
  #@markdown Diffuser models that are auto imported are NOT selected by default and must be chosen from the menu in app.

  #@markdown ### Notes about connection types.
  #@markdown #### Localtunnel = Once it gets going it is quite stable but often has "502" Errors, You must wait for THEM to fix it, please try another connection type.
  #@markdown #### Serveo = Almost, always connects however, it can drop your connection if a HTML error occurs, Simply wait for the images to finish generating, stop an re-start this step.
  #@markdown #### NGROK = Very stable but requires a token, see the "configuration" step for more details.

  #@markdown ---

  %cd {file_path}
  import os

  if connection_type == "Serveo":
    !ssh -o StrictHostKeyChecking=no -o ServerAliveInterval=60 -R 80:localhost:9090 serveo.net & . {file_path}/.venv/bin/activate; invokeai-web --outdir={outpath}

  if connection_type == "Localtunnel":
    print("How to connect to localtunnel:");
    print("A localtunnel Interface connection is generated here, To use this, please do the following ")
    print("1. Copy this IP address")
    !curl ipv4.icanhazip.com
    print("2. Click the random 'https://XXX-YYY-ZZZ.loca.lt' link that is generated below.")
    print("3. Paste the IP into the provided box and submit. ")
    print(" ")
    print("Note: An error of '502 Bad Gateway' typically is an error at Localtunnels end. A '504 Gateway Time-out' Error means invokeai has not started yet.")
    print(" ")
    !npm install -g localtunnel
    !npx localtunnel --port 9090 & . {file_path}/.venv/bin/activate; invokeai-web --outdir={outpath}

  if connection_type == "NGROK":
    if ngrok_token == "None":
      print("You have Selected NGROK but did not supply an NGROK token.")
      print("Falling back to a 'Serveo' connection type.")
      print("Please either add an NGROK token to step 1, re-run step 1, then re-run this step, or just re-run this step to use 'Servio'.")
      connection_type = "Serveo"
    if ngrok_token != "None":
      !pip install pyngrok --quiet
      from pyngrok import ngrok
      ngrok.kill()
      ngrok.set_auth_token(ngrok_token)
      public_url = ngrok.connect(9090).public_url
      print(f'InvokeAI Public URL: {public_url}')
      ! . {file_path}/.venv/bin/activate; invokeai-web --outdir={outpath}

In [ ]:
#@title 6. InvokeAI Textual Inversion Training. ALL OPTIONS ARE MANDATORY AND WILL BE CUSTOM TO YOU! { display-mode: "form" }
if ti_mode == "Yes":

  #@markdown # Textual Inversion Training.<br>
  #@markdown ## Please place the files in either .jpg/.jpeg or .png into the folder relavent to your "type" selected above, in your OUTPUT location "invokeaiOutputV3/text-inversion-training-data/modelname" where the modelname is EXACTLY what you have set below. (capitlisation included) <br><br>

  # TPU does not work, commenting out comment about it.
  # #@markdown ## Allthough not required, it is recomended to swap to a TPU if you are NOT using "Mixed precision", as this will be aproximatly 2.5X faster. You can do this in "Edit" > "Notebook Settings" > "TPU". <br><br>

  #@markdown ## Model Name.
  #@markdown Textual Inversion Name, this is both what you will need to include in the prompt to use this model and part of the file path used by this process.
  model_name_ti = "MODEL_NAME" #@param ['MODEL_NAME'] {allow-input: true}

  #@markdown ## Base Model.
  #@markdown What model should the Textual Inversion be based on? This must match the name in InvokeAI EXACTLY, captiliation included.
  modelname_ti = "stable-diffusion-v1-5" #@param ['stable-diffusion-v1-5'] {allow-input: true}

  #@markdown ## Number of Steps
  #@markdown Set the maximum training steps, The more steps the longer it takes to run however the more accurate it will be.<br>
  #@markdown Any checkpoint can be used
  steps_ti = "5000" #@param ['1000','3000','5000','10000'] {allow-input: true}

  #@markdown ## Checkpoint Frequency.
  #@markdown How often should a checkpoint be made? The more often a checkpoint is made the more space this takes up, however the less progress is lost if this crashes or is stopped.
  checkpoint_frequency = "250" #@param ['10','100','250','500','1000'] {allow-input: true}

  #@markdown ## Resolution.
  #@markdown Model resolution - This should be the same as the base model, and is a square e.g. 512 = 512X512
  resolution_ti = "512" #@param ['512','768','1024'] {allow-input: true}

  #@markdown ## Learnable property.
  #@markdown What do you want to take from the images, Style or Object?
  learnable_property = "style" #@param ['style','object'] {allow-input: false}

  #@markdown ## Mixed Presision
  #@markdown Allow "Vae Precision" of "fp16"
  mixed_precision_ti = "fp16" #@param ['no','fp16'] {allow-input: false}

  #@markdown ---
  #@markdown ## Currently there seems to be an issue where resuming a run does not work (it simply starts again, but looks like it is resuming) <br>
  #@markdown This "Works" and the settings are retained for when it is fixed at invokeai's end. <br><br>

  #@markdown ## Resume an Old Traning Run.
  #@markdown Resume training from an existing training run? If this is set to "Yes" and no pre-exisiting run with this name has reached a checkpoint, this will not run.
  continue_ti = "No" #@param ['Yes','No'] {allow-input: false}

  #@markdown ## Resume From What Checkpoint Number? - This setting is Ignored if "continue_ti" is "No"
  #@markdown This is the number of the checkpoint you would like to resume from, in the ouput directory will be files called "checkpoint-XXX" choose the highest number.<br>
  checkpoint_resume = "500" #@param ['250'] {allow-input: true}



  #Make directorys if they dont exist
  directory_path = os.path.join(outpath, "text-inversion-output", model_name_ti)
  if not os.path.exists(directory_path):
      os.makedirs(directory_path)
  directory_path = os.path.join(outpath, "text-inversion-training-data", model_name_ti)
  if not os.path.exists(directory_path):
    os.makedirs(directory_path)

  #Run the Textual Inversion Training First run
  if continue_ti == "No":
    ! . {file_path}/.venv/bin/activate; invokeai-ti --model={modelname_ti} --resolution={resolution_ti} --learnable_property={learnable_property} --initializer_token='*' --placeholder_token='<{model_name_ti}>' --train_data_dir={outpath}/text-inversion-training-data/{model_name_ti}/ --output_dir={outpath}/text-inversion-output/{model_name_ti}/ --train_batch_size=4 --gradient_accumulation_steps=1 --max_train_steps={steps_ti} --learning_rate=0.0005 --scale_lr  --lr_scheduler=constant --mixed_precision={mixed_precision_ti} --only_save_embeds --checkpointing_steps {checkpoint_frequency} --save_steps {checkpoint_frequency}

  #Run the Textual Inversion Training in Continue mode
  if continue_ti == "Yes":
    ! . {file_path}/.venv/bin/activate; invokeai-ti --model={modelname_ti} --resolution={resolution_ti} --learnable_property={learnable_property} --initializer_token='*' --placeholder_token='<{model_name_ti}>' --train_data_dir={outpath}/text-inversion-training-data/{model_name_ti}/ --output_dir={outpath}/text-inversion-output/{model_name_ti}/ --train_batch_size=1 --gradient_accumulation_steps=4 --max_train_steps={steps_ti} --learning_rate=0.0005 --scale_lr  --lr_scheduler=constant --mixed_precision={mixed_precision_ti} --only_save_embeds --checkpointing_steps {checkpoint_frequency} --save_steps {checkpoint_frequency} --resume_from_checkpoint="checkpoint-{checkpoint_resume}"
